In [46]:
import numpy as np
from tabulate import tabulate
from sympy import Symbol, solve

In [47]:
A = np.array([[0.8894, 0.0000, -0.2323, 0.1634, 0.2723],
              [-0.0545, 0.5808, 0.0000, -0.1107, 0.0363],
              [0.0182, -0.1634, 1.0527, 0.0200, 0.0635],
              [0.0545, 0.0000, -0.1325, 1.0527, 0.0000],
              [0.0363, -0.0545, 0.2632, -0.0218, 0.7623]])

In [48]:
print(tabulate(A, tablefmt="grid", floatfmt=".5f"))

+----------+----------+----------+----------+---------+
|  0.88940 |  0.00000 | -0.23230 |  0.16340 | 0.27230 |
+----------+----------+----------+----------+---------+
| -0.05450 |  0.58080 |  0.00000 | -0.11070 | 0.03630 |
+----------+----------+----------+----------+---------+
|  0.01820 | -0.16340 |  1.05270 |  0.02000 | 0.06350 |
+----------+----------+----------+----------+---------+
|  0.05450 |  0.00000 | -0.13250 |  1.05270 | 0.00000 |
+----------+----------+----------+----------+---------+
|  0.03630 | -0.05450 |  0.26320 | -0.02180 | 0.76230 |
+----------+----------+----------+----------+---------+


In [49]:
A = A.T @ A

In [50]:
print(tabulate(A, tablefmt="grid", floatfmt=".5f"))

+----------+----------+----------+----------+----------+
|  0.79862 | -0.03661 | -0.18512 |  0.20831 |  0.26903 |
+----------+----------+----------+----------+----------+
| -0.03661 |  0.36700 | -0.18636 | -0.06637 | -0.03084 |
+----------+----------+----------+----------+----------+
| -0.18512 | -0.18636 |  1.24897 | -0.16212 |  0.20423 |
+----------+----------+----------+----------+----------+
|  0.20831 | -0.06637 | -0.16212 |  1.14801 |  0.02513 |
+----------+----------+----------+----------+----------+
|  0.26903 | -0.03084 |  0.20423 |  0.02513 |  0.66060 |
+----------+----------+----------+----------+----------+


In [51]:
def danilevsky_method_v0(A: np.ndarray):
    X = A.copy()
    n = X.shape[0]
    s = np.eye(n)
    n -= 1
    for i in np.arange(n):
        ones_left = np.eye(n + 1)
        ones_left[n - 1 - i] = X[n - i]
        ones_right = np.linalg.inv(ones_left)
        X = ones_left @ X @ ones_right
        s = s @ ones_right
    p = X[0]
    x = Symbol('x')
    eigenvalues = solve(x**5 - p[0] * x**4 - p[1] * x**3 - p[2] * x**2 - p[3] * x - p[4], x)
    eigenvalues = np.array([complex(sol.evalf()) for sol in eigenvalues])
    eigenvalues = eigenvalues[np.isreal(eigenvalues)].real
    r1 = p[0] - np.trace(A)
    detA = np.linalg.det(A)
    r2 = p[n] - detA
    max_eigenvalue = np.max(eigenvalues)
    y = np.array([max_eigenvalue ** i for i in np.arange(n, -1, -1)])
    eigenvector_max_eigenvalue = s @ y
    r_eigenvector = A @ eigenvector_max_eigenvalue - max_eigenvalue * eigenvector_max_eigenvalue
    r_eigenvalue = np.sum([p[n-i] * (max_eigenvalue ** i) for i in range(n+1)]) - max_eigenvalue ** (n+1)
    return X, p, r1, r2, max_eigenvalue, eigenvector_max_eigenvalue, r_eigenvalue, r_eigenvector

def pretty_print(X, p, r1, r2, eigenvalue, eigenvector, r_eigenvalue, r_eigenvector, c=None):
    p *= -1
    x = [f"x^{i}" if i != 1 else "x" for i in range(5, 0, -1)]
    polynom = "p(x)="
    for i, j in zip(x, p):
        polynom += i
        polynom += f" - {-round(j, 3)}" if np.sign(j) == -1 else f" + {round(j, 3)}"
    print(f'1)Frobenius normal form:\n{tabulate(X, tablefmt="grid", floatfmt=".3f")}\n',\
        f'2)Characteristic polynomial:\n{polynom}\n',\
        f'3)r1 = p1 - SpA = {r1:.3e}\n',\
        f'4)r2 = p5 - detA = {r2:.3e}\n',\
        f'5)max eigenvalue: {eigenvalue}\n',\
        f'6)eigenvector:\n{tabulate([eigenvector], tablefmt="grid", floatfmt=".5f")}\n',\
        f'7)r of eigenvalue: {r_eigenvalue}\n',\
        f'8)r of eigenvector:\n{tabulate([r_eigenvector], tablefmt="grid", floatfmt=".3e")}\n',\
          "" if c is None else f'9)count of iterations: {c}')


In [52]:
X, p, r1, r2, max_eigenvalue, eigenvector_max_eigenvalue, r_eigenvalue, r = danilevsky_method_v0(A)
pretty_print(X, p, r1, r2, max_eigenvalue, eigenvector_max_eigenvalue, r_eigenvalue, r)

1)Frobenius normal form:
+--------+--------+--------+--------+--------+
| -4.223 |  6.614 | -4.722 |  1.511 | -0.174 |
+--------+--------+--------+--------+--------+
|  1.000 |  0.000 |  0.000 | -0.000 | -0.000 |
+--------+--------+--------+--------+--------+
| -0.000 |  1.000 | -0.000 | -0.000 | -0.000 |
+--------+--------+--------+--------+--------+
| -0.000 |  0.000 |  1.000 | -0.000 | -0.000 |
+--------+--------+--------+--------+--------+
|  0.000 | -0.000 |  0.000 |  1.000 | -0.000 |
+--------+--------+--------+--------+--------+
 2)Characteristic polynomial:
p(x)=x^5 - 4.223x^4 + 6.614x^3 - 4.722x^2 + 1.511x - 0.174
 3)r1 = p1 - SpA = 0.000e+00
 4)r2 = p5 - detA = -4.163e-16
 5)max eigenvalue: 1.4875053959422158
 6)eigenvector:
+----------+----------+----------+----------+---------+
| -5.44620 | -1.34189 | 12.10141 | -8.78417 | 1.00000 |
+----------+----------+----------+----------+---------+
 7)r of eigenvalue: 7.993605777301127e-15
 8)r of eigenvector:
+------------+----------

In [61]:
def danilevsky_method(A: np.ndarray):
    X = A.copy()
    n = X.shape[0]
    s = np.eye(n)
    n -= 1
    for i in np.arange(n):
        ones_left = np.eye(n+1)
        ones_left[n-1-i] = X[n-i]
        ones_right = ones_left.copy()
        ones_right[n-1-i] /= -X[n-i, n-1-i]
        ones_right[n-1-i, n-1-i] = 1 / X[n-i, n-1-i]
        X = ones_left @ X @ ones_right
        s = s @ ones_right
    p = X[0]
    r1 = p[0] - np.trace(A)
    detA = np.linalg.det(A)
    r2 = p[n] - detA
    return X, r1, r2, s, p

def power_iteration(A: np.ndarray, num_iter: int=1000, tol: float=1e-5):
    n = A.shape[0]
    u = np.zeros(n)
    u[0] = 1
    prev_eigenvalue = 0
    c = 0
    for k in range(1, num_iter + 1):
        c += 1
        v = A @ u
        v_norm = np.linalg.norm(v, np.inf)
        u = v / v_norm
        eigenvalue = v_norm
        if abs(eigenvalue - prev_eigenvalue) < tol:
            break
        prev_eigenvalue = eigenvalue
    return eigenvalue, u, c


def danilevsky_power_method(A: np.ndarray, type_eigenvector: str ='danilevsky'):
    A_copy = A.copy()
    n = A_copy.shape[0]
    X, r1, r2, s, p = danilevsky_method(A_copy)
    eigenvalue, u, c = power_iteration(A_copy)
    r_eigenvalue = np.sum([p[n-1-i] * (eigenvalue ** i) for i in range(n)]) - eigenvalue ** (n)
    if type_eigenvector == 'danilevsky':
        y = np.array([eigenvalue ** i for i in np.arange(n-1, -1, -1)])
        eigenvector = s @ y
        r_eigenvector = A_copy @ eigenvector - eigenvalue * eigenvector
    elif type_eigenvector == 'power':
        r_eigenvector = A_copy @ u - eigenvalue * u
        eigenvector = u
    else:
        raise AttributeError("Invalid type_eigenvector. Choose 'danilevsky' or 'power'.")
    return X, p, r1, r2, c, eigenvalue, eigenvector, r_eigenvalue, r_eigenvector

In [62]:
X, p, r1, r2, c, eigenvalue, eigenvector, r_eigenvalue, r_eigenvector = danilevsky_power_method(A)
pretty_print(X, p, r1, r2, eigenvalue, eigenvector, r_eigenvalue, r_eigenvector, c)

1)Frobenius normal form:
+--------+--------+--------+--------+--------+
| -4.223 |  6.614 | -4.722 |  1.511 | -0.174 |
+--------+--------+--------+--------+--------+
|  1.000 |  0.000 |  0.000 |  0.000 |  0.000 |
+--------+--------+--------+--------+--------+
|  0.000 |  1.000 |  0.000 |  0.000 |  0.000 |
+--------+--------+--------+--------+--------+
|  0.000 | -0.000 |  1.000 | -0.000 |  0.000 |
+--------+--------+--------+--------+--------+
|  0.000 | -0.000 |  0.000 |  1.000 | -0.000 |
+--------+--------+--------+--------+--------+
 2)Characteristic polynomial:
p(x)=x^5 - 4.223x^4 + 6.614x^3 - 4.722x^2 + 1.511x - 0.174
 3)r1 = p1 - SpA = 0.000e+00
 4)r2 = p5 - detA = -2.776e-17
 5)max eigenvalue: 1.4875366314428213
 6)eigenvector:
+----------+----------+----------+----------+---------+
| -5.44735 | -1.34234 | 12.10333 | -8.78681 | 1.00000 |
+----------+----------+----------+----------+---------+
 7)r of eigenvalue: -7.712578678820137e-06
 8)r of eigenvector:
+-----------+----------

In [63]:
X, p, r1, r2, c, eigenvalue, eigenvector, r_eigenvalue, r_eigenvector = danilevsky_power_method(A, type_eigenvector='power')
pretty_print(X, p, r1, r2, eigenvalue, eigenvector, r_eigenvalue, r_eigenvector, c)

1)Frobenius normal form:
+--------+--------+--------+--------+--------+
| -4.223 |  6.614 | -4.722 |  1.511 | -0.174 |
+--------+--------+--------+--------+--------+
|  1.000 |  0.000 |  0.000 |  0.000 |  0.000 |
+--------+--------+--------+--------+--------+
|  0.000 |  1.000 |  0.000 |  0.000 |  0.000 |
+--------+--------+--------+--------+--------+
|  0.000 | -0.000 |  1.000 | -0.000 |  0.000 |
+--------+--------+--------+--------+--------+
|  0.000 | -0.000 |  0.000 |  1.000 | -0.000 |
+--------+--------+--------+--------+--------+
 2)Characteristic polynomial:
p(x)=x^5 - 4.223x^4 + 6.614x^3 - 4.722x^2 + 1.511x - 0.174
 3)r1 = p1 - SpA = 0.000e+00
 4)r2 = p5 - detA = -2.776e-17
 5)max eigenvalue: 1.4875366314428213
 6)eigenvector:
+---------+---------+----------+---------+----------+
| 0.45014 | 0.11087 | -1.00000 | 0.72603 | -0.08257 |
+---------+---------+----------+---------+----------+
 7)r of eigenvalue: -7.712578678820137e-06
 8)r of eigenvector:
+------------+-----------+---

In [56]:
import numpy as np
from tabulate import tabulate
from sympy import Symbol, solve

A = np.array([[0.8894, 0.0000, -0.2323, 0.1634, 0.2723],
              [-0.0545, 0.5808, 0.0000, -0.1107, 0.0363],
              [0.0182, -0.1634, 1.0527, 0.0200, 0.0635],
              [0.0545, 0.0000, -0.1325, 1.0527, 0.0000],
              [0.0363, -0.0545, 0.2632, -0.0218, 0.7623]])

A = A.T @ A

def pretty_print(X, p, r1, r2, eigenvalue, eigenvector, r_eigenvalue, r_eigenvector, c=None):
    p *= -1
    x = [f"x^{i}" if i != 1 else "x" for i in range(5, 0, -1)]
    polynom = "p(x)="
    for i, j in zip(x, p):
        polynom += i
        polynom += f" - {-round(j, 3)}" if np.sign(j) == -1 else f" + {round(j, 3)}"
    print(f'1)Frobenius normal form:\n{tabulate(X, tablefmt="grid", floatfmt=".3f")}\n',\
        f'2)Characteristic polynomial:\n{polynom}\n',\
        f'3)r1 = p1 - SpA = {r1:.3e}\n',\
        f'4)r2 = p5 - detA = {r2:.3e}\n',\
        f'5)max eigenvalue: {eigenvalue}\n',\
        f'6)eigenvector:\n{tabulate([eigenvector], tablefmt="grid", floatfmt=".5f")}\n',\
        f'7)r of eigenvalue: {r_eigenvalue}\n',\
        f'8)r of eigenvector:\n{tabulate([r_eigenvector], tablefmt="grid", floatfmt=".3e")}\n',\
          "" if c is None else f'9)count of iterations: {c}')

def danilevsky_method(A: np.ndarray):
    X = A.copy()
    n = X.shape[0]
    s = np.eye(n)
    n -= 1
    for i in np.arange(n):
        ones_left = np.eye(n+1)
        ones_left[n-1-i] = X[n-i]
        ones_right = ones_left.copy()
        ones_right[n-1-i] /= -X[n-i, n-1-i]
        ones_right[n-1-i, n-1-i] = 1 / X[n-i, n-1-i]
        X = ones_left @ X @ ones_right
        s = s @ ones_right
    p = X[0]
    r1 = p[0] - np.trace(A)
    detA = np.linalg.det(A)
    r2 = p[n] - detA
    return X, r1, r2, s, p

def power_iteration(A: np.ndarray, num_iter: int=1000, tol: float=1e-5):
    n = A.shape[0]
    u = np.zeros(n)
    u[0] = 1
    prev_eigenvalue = 0
    c = 0
    for k in range(1, num_iter + 1):
        c += 1
        v = A @ u
        v_norm = np.linalg.norm(v, np.inf)
        u = v / v_norm
        eigenvalue = v_norm
        if abs(eigenvalue - prev_eigenvalue) < tol:
            break
        prev_eigenvalue = eigenvalue
    return eigenvalue, u, c

def danilevsky_power_method(A: np.ndarray, type_eigenvector='danilevsky'):
    A_copy = A.copy()
    n = A_copy.shape[0]
    X, r1, r2, s, p = danilevsky_method(A_copy)
    eigenvalue, u, c = power_iteration(A_copy)
    r_eigenvalue = np.sum([p[n-1-i] * (eigenvalue ** i) for i in range(n)]) - eigenvalue ** (n)
    if type_eigenvector == 'danilevsky':
        y = np.array([eigenvalue ** i for i in np.arange(n-1, -1, -1)])
        eigenvector = s @ y
        r_eigenvector = A_copy @ eigenvector - eigenvalue * eigenvector
    else:
        r_eigenvector = A_copy @ u - eigenvalue * u
        eigenvector = u
    return X, p, r1, r2, c, eigenvalue, eigenvector, r_eigenvalue, r_eigenvector

X, p, r1, r2, c, eigenvalue, eigenvector, r_eigenvalue, r_eigenvector = danilevsky_power_method(A)
pretty_print(X, p, r1, r2, eigenvalue, eigenvector, r_eigenvalue, r_eigenvector, c)


1)Frobenius normal form:
+--------+--------+--------+--------+--------+
| -4.223 |  6.614 | -4.722 |  1.511 | -0.174 |
+--------+--------+--------+--------+--------+
|  1.000 |  0.000 |  0.000 |  0.000 |  0.000 |
+--------+--------+--------+--------+--------+
|  0.000 |  1.000 |  0.000 |  0.000 |  0.000 |
+--------+--------+--------+--------+--------+
|  0.000 | -0.000 |  1.000 | -0.000 |  0.000 |
+--------+--------+--------+--------+--------+
|  0.000 | -0.000 |  0.000 |  1.000 | -0.000 |
+--------+--------+--------+--------+--------+
 2)Characteristic polynomial:
p(x)=x^5 - 4.223x^4 + 6.614x^3 - 4.722x^2 + 1.511x - 0.174
 3)r1 = p1 - SpA = 0.000e+00
 4)r2 = p5 - detA = -2.776e-17
 5)max eigenvalue: 1.4875366314428213
 6)eigenvector:
+----------+----------+----------+----------+---------+
| -5.44735 | -1.34234 | 12.10333 | -8.78681 | 1.00000 |
+----------+----------+----------+----------+---------+
 7)r of eigenvalue: -7.712578678820137e-06
 8)r of eigenvector:
+-----------+----------